# MedX Engine, Radiography and Tomography Processing
---

## 0.0. Install Libraries 

In [ ]:
import Radiography_Library as RadLib
RadLib.Install_Libraries()

---

## 0.1. Attenuation Coefficient Bisection Algorithm

### 0.1.1. Fixed Energy Step

In [ ]:
import Radiography_Library as RadLib

threads = 10

save_folder = ''

outputcsv_name = '.csv'

root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

energies = [100000, 101000, 1000] # initial energy, final energy, energy step (in eV)

tolerance = 8

RadLib.BisectionFixedEnergyStep(save_folder, threads, outputcsv_name, root_structure, energies, tolerance)

### 0.1.2. Energies from NIST CSV

In [ ]:
import Radiography_Library as RadLib

threads = 10

save_folder = ''

input_csv = 'CdTe_nist.csv'

outputcsv_name = '.csv'
root_structure = ['Transportation', 'Ratio', 'Mass_Attenuation'] # tree name, branch 1, branch 2

tolerance = 8

RadLib.BisectionEnergiesNIST(save_folder, threads, outputcsv_name, root_structure, input_csv, tolerance)

### 0.1.3. Plot Attenuation Results

In [ ]:
import Radiography_Library as RadLib

save_folder = 'BUILD/ROOT/'

DATA = \
[
{"CSV": "CadTel_map.csv", "X": "Energy", "Y": "AtCoefficient", "LABEL": "CadTel", "MARKER": "o", "MARKERSIZE": 1, "COLOR": "red", "ALPHA": 0.5},
{"CSV": "CdTe_nist.csv",  "X": "Energy", "Y": "AtCoeff"      , "LABEL": "NIST",   "MARKER": "o", "MARKERSIZE": 1, "COLOR": "red", "ALPHA": 0.5},
]

title   = r"Element ($$)"
x_label = r"Energy ($keV$)"
y_label = r"Mass Attenuation Coefficient ($cm^2/g$)"

X_axis_log = True
Y_axis_log = True

Figure_Text = None

Font_Size_Normal = 16
Font_Size_Large  = 20

save_as = None  # '.png'

RadLib.Plot_Att_Coeff(save_folder, DATA, title, x_label, y_label, X_axis_log, Y_axis_log, Figure_Text, Font_Size_Normal, Font_Size_Large, save_as)

---

## 1.1. Run Radiography

In [ ]:
import Radiography_Library as RadLib

threads  = 1
energy   = 80      # keV
sim_time = 4       # min
iteration_time = 1 # min

spectra_mode        = '80kvp' # 'mono (0)' or '80kvp (1) or '140kvp (2)'
detector_parameters = {'nColumns': 1, 'nRows': 1}
gun_parameters      = {'X': 0, 'Y': 0, 'gaussX': 'true', 'SpanX': 230, 'SpanY': 240}

RadLib.RunRadiography(threads, energy, sim_time, iteration_time, spectra_mode, detector_parameters, gun_parameters, alarm=False)

## 1.2. Run DEXA

In [ ]:
import Radiography_Library as RadLib

threads  = 10
sim_time = 20#8*60     # min
iteration_time = 60 # min

spectra_mode = 'poly' # 'mono (0)' or 'poly (1)'

RadLib.RunDEXA(threads, sim_time, iteration_time, spectra_mode, detector_parameters=None, gun_parameters=None, alarm=False)

## 1.3. Merge Root Files

In [ ]:
import Radiography_Library as RadLib

save_folder = 'BUILD/ROOT/'
save_folder = '/Users/miguelcomett/GEANT4/G4_MedX'

starts_with = 'Rad_40'
starts_with = 'Rad_80'
starts_with = 'Poly_140kvp'
starts_with = 'Poly_80kvp'
starts_with = 'CT'

output_name = starts_with

# trim_coords = (-200, 200, -200, 200)  # x_min, x_max, y_min, y_max. (slower method)
trim_coords = None

RadLib.Merge_Roots_HADD(save_folder, starts_with, output_name, trim_coords) 

## 1.4. Get ROOT Summary Data & Histograms 

In [ ]:
import Radiography_Library as RadLib

save_folder = 'BUILD/ROOT/'
save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

root_file = 'Rad_40'
root_file = 'Rad_80'
root_file = 'Poly_80kvp.root'
root_file = 'Poly_140kvp.root'

# root_file = ''

hits_tree      = 'Hits';                hits_branches      = ['x_ax', 'y_ax']
summary_tree   = 'Run Summary';         summary_branches   = ['Number_of_Photons', 'Sample_Mass_kg', 'EDep_Value_TeV', 'Radiation_Dose_uSv']
radiation_tree = 'Radiation Dose';      radiation_branches = ['Tissue', 'Radiation_Dose_uSv']
spectra_tree   = 'Energy Spectra keV';  spectra_branches   = ['Energies', 'Counts']

RadLib.Summary_Data(save_folder, root_file, hits_tree, hits_branches, summary_tree, summary_branches, 
                    radiation_tree, radiation_branches, spectra_tree, spectra_branches)

range_x = [-250, 250, 1000] # range_min, range_max, bins (mm) 
range_y = [-250, 250, 1000] # range_min, range_max, bins (mm)
range_spectra = [0, 150, 300] # range_min, range_max, bins (keV)

RadLib.XY_1D_Histogram(save_folder, root_file, hits_tree, hits_branches, spectra_tree, spectra_branches, range_x, range_y, range_spectra)

---

## 2. Root File to Heatmap

In [ ]:
import Radiography_Library as RadLib

save_folder = 'BUILD/ROOT/'
save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

root_file = 'Rad_40.root'
root_file = 'Rad_80.root'
root_file = 'Poly_80kvp.root'
root_file = 'Poly_140kvp.root'

root_structure = {"tree_name": "Photons", "x_branch": "X_axis", "y_branch": "Y_axis"}
root_structure = {"tree_name": "Hits", "x_branch": "x_ax", "y_branch": "y_ax"}
dimensions  = {"len_X": 300, "len_Y": 200, "shift_X": 0, "shift_Y": -150} # mm 
dimensions  = {"len_X": 300, "len_Y": 300, "shift_X": 0, "shift_Y": 0} # mm

pixel_size = 0.5 # mm

heatmap_raw = RadLib.Root_to_Heatmap(save_folder, root_file, root_structure, dimensions, pixel_size, progress_bar=True)
heatmap = RadLib.Logarithmic_Transform(heatmap_raw)

RadLib.Plotly_from_memory(projection = heatmap, size = [600, 600])
# RadLib.Plot_Heatmap(heatmap, save_as='')

In [ ]:
RadLib.Plotly_from_memory(projection = heatmap, size = [600, 600])

---

## 3. DEXA: Tissue Segmentation

### Calculate Heatmaps

In [ ]:
import Radiography_Library as RadLib

save_folder = 'BUILD/ROOT/'
save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

save_folder = '/Volumes/MCF Samsung/Geant4'

rootnames = ["Rad_40.root", "Rad_80.root"]
rootnames = ['Poly_80kvp.root', 'Poly_140kvp.root']

root_structure = {"tree_name": "Hits", "x_branch": "x_ax", "y_branch": "y_ax"}
tree_name = "Hits"; x_branch  = "x_ax"; y_branch  = "y_ax"

dimensions  = {"len_X": 260, "len_Y": 260, "shift_X": 0, "shift_Y": 0} 

pixel_size = 0.5

low_energy_img  = RadLib.Root_to_Heatmap(save_folder, rootnames[0], root_structure, dimensions, pixel_size, progress_bar=True)
high_energy_img = RadLib.Root_to_Heatmap(save_folder, rootnames[1], root_structure, dimensions, pixel_size, progress_bar=True)
low_energy_img  = RadLib.Logarithmic_Transform(low_energy_img)
high_energy_img = RadLib.Logarithmic_Transform(high_energy_img)

In [ ]:
dif_projection = high_energy_img - low_energy_img
dif_projection = low_energy_img - high_energy_img
RadLib.Plot_Heatmap(dif_projection, save_as='')

### Save heatmaps as CSV

In [ ]:
import Radiography_Library as RadLib

read_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

RadLib.Save_Heatmap_to_CSV(low_energy_img,  read_folder, save_as = '40kev_Projection')
RadLib.Save_Heatmap_to_CSV(high_energy_img, read_folder, save_as = '80kev_Projection')

### Read heatmap's CSV

In [ ]:
import Radiography_Library as RadLib

read_folder = '/Users/miguelcomett/GEANT4/G4_MedX/'

low_energy_img  = RadLib.Read_Heatmap_from_CSV(read_folder, '40kev_Projection')
high_energy_img = RadLib.Read_Heatmap_from_CSV(read_folder, '80kev_Projection')

### Perform Segmentation

In [ ]:
import Radiography_Library as RadLib

sigma = 0
wn = 0

save_as = ['', '', '', '', '', '', '', ''] # Low Energy, High Energy, Bone [SLS], Tissue [SLS], Bone [SSH], Tissue [SSH], Bone [ACNR], Tissue [ACNR]

SLS_Bone, SLS_Tissue, SSH_Bone, SSH_Tissue, ACNR_Bone, ACNR_Tissue = RadLib.IsolateTissues(low_energy_img, high_energy_img, sigma, sigma, wn, '', save_as, save_all='')

---

## 4. Bone Mineral Density (BMD)

In [ ]:
import Radiography_Library as RadLib

thickness_bone, thickness_tissue = RadLib.Bone_Mineral_Density(SLS_Bone, SLS_Tissue)
RadLib.Plot_Heatmap(heatmap, '')
RadLib.Plot_Plotly(thickness_bone, xlim, ylim)

---

## 5.1. Calculate Interactive CNR

#### Trim Image

In [ ]:
from PIL import Image; import matplotlib.pyplot as plt

save_folder = 'RESULTS/'
image = 'ssh' + '.png'
image = Image.open(save_folder + image)
image = image.convert('L')

print(image.size)
width = image.width; height = image.height

trim = 200
# image = image.crop((trim, trim, width - trim, height - trim)) # left, top, right, bottom
# image = image.crop((8410, trim, width - 60, height - trim))

# plt.imshow(image, cmap='gray'); plt.axis('off'); plt.show()

#### Launch Interactive CNR

In [ ]:
%matplotlib widget 
%matplotlib tk
import Radiography_Library as RadLib

RadLib.Interactive_CNR(image)

## 5.2. Calculate Fixed CNR 

In [ ]:
import Radiography_Library as RadLib

image_path = "RESULTS/" + "a" + ".png"
save_as = ''

shftx_s = 0.0 # shift x-coordinate signal box
shfty_s = 0.0 
shftx_b = 200.0 # shift x-coordinate background box
shfty_b = 0.0

coords_signal  = [1200 + shftx_s, 1000 + shfty_s, 1800 + shftx_s, 1800 + shfty_s] # x1, y1, x2, y2
coords_bckgrnd = [2100 + shftx_b, 1000 + shfty_b, 2300 + shftx_b, 1800 + shfty_b] # x1, y1, x2, y2

RadLib.Fixed_CNR(image_path, save_as, coords_signal, coords_bckgrnd)

---

## 6.1. Denoise with Skimage.Denoise_Bilateral

In [ ]:
import Radiography_Library as RadLib

# load array
path = SSH_Bone
isArray = True

# or load image
if isArray == False:
    save_folder = 'RESULTS/'
    path = save_folder + 'a' + '.png'

sigma_color = 0.05
sigma_spatial = 20

Denoised_Image = RadLib.Denoising_Auto_Edge_Detection(path, isArray)

## 6.2. Denoise by Fourier Transform

In [ ]:
import Radiography_Library as RadLib

array = heatmap
isHann = False

alpha = 1

save_as = ''
isCrossSection = False # plot crosss-section

fft_image = RadLib.Denoising_Window(array, isHann, alpha, save_as, isCrossSection)

---

## 7. Save Plotly Heatmap

In [ ]:
import Radiography_Library as RadLib

# array = htmp_array
# array = Denoised_Image
array = ACNR_Bone
array = low_energy_img

title   = r"$ \large{ \text{Thorax Radiography Projection(40 keV)} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

width  = 800
height = 800

save_as = ''

RadLib.Plotly_Heatmap_1(array, xlim, ylim, title, x_label, y_label, width, height, save_as)

#### Plot with annotation and rectangles

In [ ]:
import Radiography_Library as RadLib

array = heatmap

title   = r"$ \large{ \text{Low energy projection (40 keV), 100M Photons} } $"
x_label = r"$ \large{ \text{X Axis} \ (mm)} $"
y_label = r"$ \large{ \text{Y Axis} \ (mm)} $"

sqr_1_coords = [10, 10, -10, -10]
sqr_2_coords = [10, 10, -10, -10]

annotation = 'CNR = ' 

width  = 700
height = 700

save_as = ''

RadLib.Plotly_Heatmap_2(array, xlim, ylim, title, x_label, y_label, sqr_1_coords, sqr_2_coords, annotation, width, height, save_as)

---

# 8. CT Scan

## 8.1 Run CT loop

In [ ]:
import Radiography_Library as RadLib

threads = 10
starts_with = 'CT'

angles = range(303, 314, 1)

slices = {"y0": -240, "y1": 240, "step": 4} # mm
gun_parameters = {'X': 0, 'Y': 0, 'gaussX': 'true', 'SpanX': 230, 'SpanY': 0.01}

beams_per_line = 600_000

RadLib.CT_Loop(threads, starts_with, angles, slices, gun_parameters, beams_per_line, alarm=False)

## 8.2. CT Summary

In [ ]:
import Radiography_Library as RadLib

save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/CT2/0_Roots'

summary_tree = 'Run Summary';         summary_branches = ['Number_of_Photons', 'EDep_Value_TeV', 'Radiation_Dose_uSv']
spectra_tree = 'Energy Spectra keV';  spectra_branches = ['Energies', 'Counts']
spectra_tree = None

RadLib.CT_Summary_Data(save_folder, summary_tree, summary_branches, spectra_tree, spectra_branches)

## 8.3. Calculate projections at every angle from roots and save to CSV

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT2'
save_folder = directory + '/0_Roots/'

filename = 'Aang'
root_structure = {"tree_name": "Hits", "x_branch": "x_ax", "y_branch": "y_ax"}

slices = {"start": 0, "end": 359, "step": 1}

dimensions  = {"len_X": 230, "len_Y": 240, "shift_X": 0, "shift_Y": 0} # mm 
# dimensions  = {"len_X": 110, "len_Y": 80, "shift_X": 0, "shift_Y": -170} # mm 

pixel_size = 0.5 # mm

gun_span = 230 # mm
# gun_span = 100 # mm

csv_write = directory + '/1_Processed/'

directory = '/Volumes/MCF_1TB/Geant4/CT/CT0'
save_folder = directory + '/0_Roots/'
dimensions  = {"len_X": 250, "len_Y": 75, "shift_X": 0, "shift_Y": -150} # mm 
root_structure = {"tree_name": "Photons", "x_branch": "X_axis", "y_branch": "Y_axis"}
gun_span = 250 
    
RadLib.Calculate_Projections(save_folder, filename, slices, root_structure, dimensions, pixel_size, gun_span, csv_write)

## 8.4. Load projections from CSV and perfom CT reconstruction

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT2'
directory = '/Volumes/MCF_1TB/Geant4/CT/CT0'
csv_read  = directory + '/1_Processed/'
csv_write = directory + '/1_Processed/'
 
sigma = 1.0
slices = {"start": 0, "end": 359, "step": 1}

# slices_in  = {"initial": -240, "final": 240, "step": 4} # mm
# slices_out = slices_in

# slices_in  = {"initial": 0, "final": 80, "step": 1} # mm    
# slices_out = {"initial": 0, "final": 120, "step": 1} # mm 

slices_in= {"initial": 0, "final": 75, "step": 1} # mm 
slices_out = slices_in

RadLib.RadonReconstruction(csv_read, csv_write, slices, slices_in, slices_out, sigma, write = True)

## 8.5. Export to DICOM

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT2'
directory = '/Volumes/MCF_1TB/Geant4/CT/CT0'
csv_slices = directory + '/1_Processed/'

slices = {"start": 0, "end": 120, "step": 1}
slices = {"start": 0, "end": 75, "step": 1}

# constants = {"µ_water": 0.1857, "µ_air": 0.1662, "air_tolerance": -10_000} # 80 keV
constants = {"µ_water": 0.18, "µ_air": 0.000204553, "air_tolerance": -10_000} # 80 keV

constant_factor = -0.0001 #- constants["µ_water"]
# constant_factor = - constants["µ_water"]
scale_factor = 1000/(constants["µ_water"] - constants["µ_air"])

percentile = 100

HU_images = RadLib.CoefficientstoHU(slices, csv_slices, constants, constant_factor, scale_factor, percentile)


save_folder = directory + '/2_DICOMs/'

slice_thickness = 0.5 # mm
slice_spacing = 1 # mm

# slice_thickness = 0.5 # mm
# slice_spacing = 7.5 # mm

RadLib.Export_to_Dicom(HU_images, slice_thickness, slice_spacing, save_folder, compressed=False)

--- 

### Test Plots

#### Plot 3x1

In [ ]:
import Radiography_Library as RadLib

directory = '/Users/miguelcomett/GEANT4/G4_MedX/CT1'
csv_read = directory + '/1_Processed/'

RadLib.Plotly_3x1(csv_read, step = 40)

#### Plot Projection

In [ ]:
import Radiography_Library as RadLib

save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/CT3/1_Processed/1_Heatmaps/'
filename = 'Projection_270.csv'
projection = RadLib.Plotly_from_file(save_folder, filename, size=[500, 500])

In [ ]:
import Radiography_Library as RadLib

save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/CT3/1_Processed/0_Raw_Projections'
filename = 'CT_Raw_235.csv'
projection = RadLib.Plotly_from_file(save_folder, filename, size=[500, 500])

#### Plot Slice

In [ ]:
import Radiography_Library as RadLib

save_folder = '/Users/miguelcomett/GEANT4/G4_MedX/CT3/Slices_test/'
filename = 'CT_slice_1.csv'
projection = RadLib.Plotly_from_file(save_folder, filename, size=[500, 500])